In [11]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rishikeshyadav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rishikeshyadav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())


words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [5]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [6]:
training = []
output_empty = [0] * len(classes)

In [7]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_50969/619861672.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [12]:
sdg = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sdg, metrics=['accuracy'])

In [13]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')


Epoch 1/200
46/46 [==============================] - 0s 2ms/step - loss: 4.3055 - accuracy: 0.0044
Epoch 2/200
46/46 [==============================] - 0s 4ms/step - loss: 4.2817 - accuracy: 0.0087
Epoch 3/200
46/46 [==============================] - 0s 3ms/step - loss: 4.2526 - accuracy: 0.0437
Epoch 4/200
46/46 [==============================] - 0s 834us/step - loss: 4.2239 - accuracy: 0.0437
Epoch 5/200
46/46 [==============================] - 0s 855us/step - loss: 4.2015 - accuracy: 0.0524
Epoch 6/200
46/46 [==============================] - 0s 1ms/step - loss: 4.1389 - accuracy: 0.0786
Epoch 7/200
46/46 [==============================] - 0s 2ms/step - loss: 4.0980 - accuracy: 0.0568
Epoch 8/200
46/46 [==============================] - 0s 1ms/step - loss: 4.0626 - accuracy: 0.0786
Epoch 9/200
46/46 [==============================] - 0s 862us/step - loss: 3.8586 - accuracy: 0.1266
Epoch 10/200
46/46 [==============================] - 0s 1ms/step - loss: 3.8932 - accuracy: 0.1135
Epo

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
